# **RED NEURONAL CIFRADO CESAR TAMAÑO AJUSTABLE**

In [ ]:
from keras.models import Sequential,Model
from keras.layers import SimpleRNN, Input, Dense
import numpy as np
import matplotlib.pyplot as plt 
from keras.losses import sparse_categorical_crossentropy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import string
import random

# Parametros configuración inicial
word_size = 3 # tamaño cadenas
shift = 3 # Desplazamiento letra (cifrado)

# Declaración de parámetros
nn = [120,80,120,80,26*word_size]
X_train = []
Y_train = []
X_test = []
Y_test = []
X = []
Y = []

# Generar el dataset de entrenamiento
X, Y = generate_words(10000,0,X)
X_train = wordToNumber(X)
Y_train = wordToNumber(Y)

# Generar el dataset de test (datos nuevos, no usados en el entrenamiento)
X, Y = generate_words(1000,1,X)
X_test = wordToNumber(X)
Y_test = wordToNumber(Y)

In [ ]:
def crearModelo(loss='poisson',activation='relu', unit1=120, unit2=80,optimizer='RMSprop'):

  # Definir modelo    
  model = Sequential()

  # Capas
  model.add(Dense(nn[4], input_dim=(26*word_size)))
  model.add(Dense(unit1, activation='relu'))
  model.add(Dense(unit2, activation='relu'))
  model.add(Dense(unit1, activation='relu'))
  model.add(Dense(nn[4], activation=activation))

  # Compilar modelo
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model

In [ ]:
model = crearModelo()

print(np.shape(X_train))

snn = model.fit(X_train, Y_train, epochs=120, batch_size=400)

scores = model.evaluate(X_test, Y_test)   
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


pred = model.predict(X_test, batch_size=15, verbose=1)  
print(pred) 


In [ ]:
print(numberToWord(Y_test))
print(numberToWord(pred))
print("------")

print("\n\n*************************************************")
print("El modelo acierta el " + str(precisionDescifrado(numberToWord(pred),numberToWord(Y_test))) + "% de palabras")
print("*************************************************")

print("\n*************************************************")
print("El modelo acierta el " + str(precisionDescifrado_letras(numberToWord(pred),numberToWord(Y_test))) + "% de letras")
print("*************************************************")  


print("\n\n*************************************************") 
print("PRUEBA - RESULTADO: javier")
resul = model.predict(wordToNumber(encrypted(["javier"])), batch_size=20, verbose=1)  
print(numberToWord(resul)) #RESULTADO: javier
print("*************************************************") 

In [ ]:
def generate_words (number_words,test,X_train):
  words = []

  if(test == 0):
    for x in range(number_words):
      words.append(''.join(random.SystemRandom().choice(string.ascii_letters).lower() for _ in range(word_size)))
  else:
    for x in range(number_words):
      repeated = 1
      while(repeated == 1):
        word = (''.join(random.SystemRandom().choice(string.ascii_letters).lower() for _ in range(word_size)))
        if(word not in X_train):
          words.append(word)
          repeated = 0

  encrypted_words = []

  for word in words:
    cadena = ""
    for x in range(0,word_size):
      cadena += chr((((ord(word[x]) - 97) + shift) % 26) + 97)
    encrypted_words.append(cadena)

  return encrypted_words, words

In [ ]:
def encrypted(words):
  encrypted_words = []

  for word in words:
    cadena = ""
    for x in range(0,word_size):
      cadena += chr((((ord(word[x]) - 97) + shift) % 26) + 97)
    encrypted_words.append(cadena)
  
  return encrypted_words

In [ ]:

def wordToNumber(words):

    word_id = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9,'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14,'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19,'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}
    lista = []
    x = 0
    one_hot_encoding = np.zeros((len(words),len(word_id)*word_size))
    for word in words:  
      for i in range(0,word_size):
        one_hot_encoding[x][word_id[word[i]]+(26*i)] = 1
      x+=1
    return one_hot_encoding


In [ ]:
def numberToWord(numbers):

    word_id = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j',10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}
    words = []

    for number in numbers:
      cadena = ""
      for i in range(0,word_size):
        car = '-'
        for x in range(0,len(word_id)):
          if(number[x+(i*26)] > 0):
            car =  word_id[x]
        cadena += car
      words.append(cadena)
    return words
        


In [ ]:
def precisionDescifrado(pred, Y_test):

  cont = 0

  for x in range(0,len(pred)):
    if(pred[x] == Y_test[x]):
      cont+=1

  return (cont / len(pred) * 100)
 

In [ ]:
def precisionDescifrado_letras(pred, Y_test):

  cont = 0

  for x in range(0,len(pred)):
    a = pred[x]
    b = Y_test[x]
    for i in range(0,word_size):
      if(a[i] == b[i]):
        cont+=1

  return (cont / (word_size*len(pred)) * 100)